In [1]:
from ultralytics import YOLO
import onnxruntime as ort
from utils import get_image, sigmoid
import torch
import timm
import onnx

/home/ubuntu/anaconda3/envs/ytenew/lib/python3.11/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.3.0) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(
/home/ubuntu/anaconda3/envs/ytenew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/anaconda3/envs/ytenew/lib/python3.11/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
img_cls, img_det = get_image("/home/ubuntu/duynd/AIyte/secondnd/input/train/18ee9ef3baea468de2087e0edd85e919.dicom")

In [3]:
model1 = timm.create_model("tf_efficientnet_b0", num_classes=1)

state_dict = torch.load("/home/ubuntu/ductq/yte/results/ckpt/v5/last.ckpt")["state_dict"]
state_dict = {k.replace('model.', ''):v for k, v in state_dict.items()}

model1.load_state_dict(state_dict)
model1.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm

In [4]:
with torch.no_grad():
    x = torch.tensor(img_cls)
    y = model1(x)

print(sigmoid(y))

tensor([[0.9988]])


In [5]:
onnx_model1 = onnx.load("/home/ubuntu/ductq/triton_pipeline/models/classification_model/1/model.onnx")
ort_sess1 = ort.InferenceSession('/home/ubuntu/ductq/triton_pipeline/models/classification_model/1/model.onnx')
outputs1 = ort_sess1.run(None, {'images': img_cls})
sigmoid(outputs1[0])

array([[    0.99877]], dtype=float32)

In [4]:
model2 = YOLO("/home/ubuntu/ductq/yte/runs/detect/train/weights/last.pt")
model2(torch.from_numpy(img_det)/255., device="cuda", conf=0.5, iou=0.6)
# model.eval()
# model.export(format="onnx",imgsz=512)

# torch.onnx.export(model,                                # model being run
#                   torch.randn(1, 3, 512, 512).cuda(),    # model input (or a tuple for multiple inputs)
#                   "yolo.onnx",           # where to save the model (can be a file or file-like object)
#                   input_names = ['images'],              # the model's input names
#                   output_names = ['output'])    


0: 512x512 1 Cardiomegaly, 25.8ms
Speed: 3.7ms preprocess, 25.8ms inference, 1264.3ms postprocess per image at shape (1, 3, 512, 512)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Aortic enlargement', 1: 'Cardiomegaly'}
 obb: None
 orig_img: array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0,

In [5]:
model2_onnx=YOLO("/home/ubuntu/ductq/yte/runs/detect/train/weights/last.onnx")
model2_onnx.predict(torch.tensor(img_det)/255., device="cuda", imgsz=512, conf=0.1, iou=0.6)

Loading /home/ubuntu/ductq/yte/runs/detect/train/weights/last.onnx for ONNX Runtime inference...


Preferring ONNX Runtime TensorrtExecutionProvider
*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:507 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************



2025-01-10 16:19:39.035955216 [E:onnxruntime:Default, provider_bridge_ort.cc:1848 TryGetProviderInfo_TensorRT] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1539 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_tensorrt.so with error: libnvinfer.so.10: cannot open shared object file: No such file or directory

2025-01-10 16:19:39.221347570 [E:onnxruntime:Default, provider_bridge_ort.cc:1862 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1539 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcufft.so.11: cannot open shared object file: No such file or directory

2025-01-10 16:19:39.221385594 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:993 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Require c

RuntimeError: Error when binding input: There's no data transfer registered for copying tensors from Device:[DeviceType:1 MemoryType:0 DeviceId:0] to Device:[DeviceType:0 MemoryType:0 DeviceId:0]

In [10]:
import numpy as np
import cv2

In [1]:
from ultralytics import YOLO

# model1 = YOLO("/home/ubuntu/ductq/yolo11n.pt")

/home/ubuntu/anaconda3/envs/ytenew/lib/python3.11/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.3.0) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


In [2]:
model_onnx = YOLO("/home/ubuntu/ductq/yolo11n.onnx",task='detect')

In [3]:
import torch

In [10]:
import onnxruntime as ort

# Create a session with CUDAExecutionProvider
sess = ort.InferenceSession('/home/ubuntu/ductq/yte/runs/detect/train/weights/last.onnx', \
                           providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

# Check the active execution providers
print("Active providers:", sess.get_providers())

Active providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']
